In [1]:
from helpers import *

In [2]:
import sys
sys.path.append('..')
import umbrella_mesh
import linkage_vis
import load_jsondata
from configuration import *
from visualization_helper import *

In [3]:
import numpy as np

In [4]:
sys.path.append('../UmbrellaGen')
import grid_gen

### Define the grid dimensions and Umbrella Valence you want

In [5]:
rows = 2
cols = 2 # one col is two element : /\ + \/

In [6]:
degree = 3 # TriUmbrellas
numUmbrellas = get_num_cells(rows, cols)

NameError: name 'get_num_cells' is not defined

### Define your favorite height, material and cross-section function! Umbrellas are sequenced row-wise if you use the grid

In [ ]:
height_scales = height_cst(rows, cols)
print(f"There is {numUmbrellas} cells with respective heights:\n{height_scales}")

### Compute Geometry with Regular Grid

In [ ]:
# The height function is a scaling function relative to the minHeight
minHeight = 64
grid_gen.genUmbrellaWithHeights(degree, rows, cols, height_scales = height_scales, minHeight = minHeight)

### Initialization of Umbrella Class

In [ ]:
name = 'grid_dump'
input_path = '../UmbrellaGen/{}.json.gz'.format(name)
io, input_data, target_mesh, curr_um, plate_thickness_scaled, target_height_multiplier = parse_input(input_path, handleBoundary = False, isHex = (degree==6), use_target_surface = False)
print(f"\
PLATE CHARACTERISTIQUES:\n\
\tplate thickness   : {input_data['thickness'], plate_thickness_scaled}\n\
\tplate edge length : {input_data['plate_edge_length']}")

### Undeployed Schema

In [ ]:
plot2D(input_data, curr_um)

### 3D Structure

In [ ]:
rod_colors = get_color_field(curr_um, input_data, uidBased = False) 
lview = linkage_vis.LinkageViewer(curr_um, width=800, height=600)
lview.update(scalarField = rod_colors)
lview.show()

### Set Target Height

In [ ]:
# Percents
active_cells    = [0, numUmbrellas-1] # cells indexes
target_percents = [0, 100] # openning percent (0 means close cell)
target_heights = percent_to_height(curr_um.umbrellaHeights, input_data['thickness'], active_cells, target_percents)

In [ ]:
# VALIDATE INPUT
assert min(active_cells)>=0, 'active cells indexes should be positive'
assert max(active_cells)<curr_um.numUmbrellas(), f'active cells indexes are too big (max accepted value:{curr_um.numUmbrellas()-1})'
assert max(target_heights)*input_data['thickness']<=max(curr_um.umbrellaHeights), f'target height is too high (max accepted value:{max(curr_um.umbrellaHeights)})'

### Deploy Structure

In [ ]:
%%time
dep_weights              = set_actives_dep_weights(curr_um.numUmbrellas(), active_cells)
target_height_multiplier = set_target_height(curr_um.numUmbrellas(), active_cells, target_heights)
success, eqays = deploy_umbrella_pin_rigid_motion(curr_um,
                                                  plate_thickness_scaled,
                                                  target_height_multiplier,
                                                  lview, 
                                                  rod_colors,
                                                  analysis = True,
                                                  dep_weights=dep_weights)
print('success : ', success, '\n')

In [ ]:
eqays.plot()

## Stress Visualization

---

In [ ]:
sview_vm = plot_stress(curr_um, 'Von Mises')
sview_vm.show()

In [ ]:
sview_bending = plot_stress('Bending', curr_um.maxBendingStresses())
sview_bending.show()

In [ ]:
sview_twisting = plot_stress('Twisting', curr_um.twistingStresses())
sview_twisting.show()